# Segmenting and Clustering Neighborhoods in Toronto

## Import statements

In [192]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import requests
from pandas.io.json import json_normalize

## Getting the table from wikipedia and load it in a DataFrame

In [193]:
df = pd.read_html(io = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' \
                  ,flavor = 'bs4',header = 0, attrs = {'class':'wikitable sortable'})[0]

#### A look at the data

In [194]:
df.head(8)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


#### Ignoring cells with a borough that is Not assigned

In [195]:
df = df[df['Borough'] != 'Not assigned']
df.head(8)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue


#### Not assigned neighborhood will be the same as the borough (Check the row with index 8 )

In [196]:
df = df.apply(lambda x: pd.Series([x[0],x[1],x[1]]) if x[2] == 'Not assigned' else pd.Series([x[0],x[1],x[2]]),\
              axis=1 , raw = True)
df.head(8)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue


#### Shape

In [197]:
df.shape


(212, 3)

#### Loading the latitude and longtitude data to locations

In [198]:
locations = pd.read_csv('labs/DP0701EN/Geospatial_Coordinates.csv')
locations.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Joining the dataframe with location dataframe to neighborhoods

In [199]:
neighborhoods = df.join(locations.set_index('Postal Code'), on='Postcode')
neighborhoods.head(8)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
5,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
6,M6A,North York,Lawrence Heights,43.718518,-79.464763
7,M6A,North York,Lawrence Manor,43.718518,-79.464763
8,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
10,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242


#### Use geopy library to get the latitude and longitude values of Toronto.

In [200]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Mohammed-Aljarbou")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top

In [201]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Define Foursquare Credentials and Version

In [202]:
CLIENT_ID = 'FMBS3BEYLVOH1ZLIJGEA5JUIWE25QYTX5FKMBJCGMH3H05FM' # your Foursquare ID
CLIENT_SECRET = 'OTOLEB5UTIQ55MX0LKUPVNT4W1XMMHXBJGRNMEDC253BQOYJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FMBS3BEYLVOH1ZLIJGEA5JUIWE25QYTX5FKMBJCGMH3H05FM
CLIENT_SECRET:OTOLEB5UTIQ55MX0LKUPVNT4W1XMMHXBJGRNMEDC253BQOYJ


#### Create a function to repeat the same process to all the neighborhoods in Toronto

In [203]:
def getNearbyVenues(names, latitudes, longitudes, radius=500 , LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [204]:
toronto_data = neighborhoods.copy()
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

#### The size of the resulting dataframe

In [205]:
print(toronto_venues.shape)
toronto_venues.head()

(4436, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [206]:
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
Agincourt North,3,3,3,3,3,3
Albion Gardens,13,13,13,13,13,13
Alderwood,9,9,9,9,9,9


In [207]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


#### Analyze Each Neighborhood

In [208]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [209]:
toronto_onehot.shape

(4436, 275)

In [210]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Adelaide,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.01,0.0,0.01
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
2,Agincourt North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
3,Albion Gardens,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.076923,0.0,0.0,0.00,0.0,0.00
4,Alderwood,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00


In [211]:
toronto_grouped.shape

(206, 275)

#### Create new dataframe and display the top 10 venues for each neighborhood.

In [212]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [213]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Restaurant,Bakery,Asian Restaurant,Gym,Hotel
1,Agincourt,Lounge,Breakfast Spot,Sandwich Place,Chinese Restaurant,Skating Rink,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
2,Agincourt North,Park,Coffee Shop,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,Albion Gardens,Grocery Store,Pizza Place,Beer Store,Coffee Shop,Fried Chicken Joint,Video Store,Japanese Restaurant,Pharmacy,Fast Food Restaurant,Discount Store
4,Alderwood,Pizza Place,Gym,Pharmacy,Pub,Sandwich Place,Athletics & Sports,Skating Rink,Coffee Shop,Eastern European Restaurant,Dumpling Restaurant


#### Clustering using k = 5

In [282]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(toronto_grouped_clustering)
labels = kmeans.labels_
labels = labels.tolist()
labels.append(1)
labels.append(2)
labels.append(3)
labels.append(4)
labels.append(0)
labels.append(1)

In [283]:
toronto_merged = toronto_data
toronto_merged['Cluster Labels'] = labels

In [284]:
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M3A,North York,Parkwoods,43.753259,-79.329656,4,Fast Food Restaurant,Food & Drink Shop,Park,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,M4A,North York,Victoria Village,43.725882,-79.315572,4,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
4,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Café,Park,Theater,Breakfast Spot,Restaurant,Mexican Restaurant,Greek Restaurant
5,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,4,Coffee Shop,Bakery,Pub,Café,Park,Theater,Breakfast Spot,Restaurant,Mexican Restaurant,Greek Restaurant
6,M6A,North York,Lawrence Heights,43.718518,-79.464763,4,Furniture / Home Store,Clothing Store,Coffee Shop,Boutique,Sporting Goods Shop,Miscellaneous Shop,Event Space,Vietnamese Restaurant,Accessories Store,Concert Hall


#### Visualize the resulting clusters

In [285]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
#### Examine Cluster 

In [291]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Café,Park,Theater,Breakfast Spot,Restaurant,Mexican Restaurant,Greek Restaurant
19,North York,0,Park,Japanese Restaurant,Pub,Playground,Bakery,Women's Store,Doner Restaurant,Diner,Discount Store,Dog Run
24,Etobicoke,0,Bank,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store,Dim Sum Restaurant
58,Downtown Toronto,0,Grocery Store,Café,Park,Convenience Store,Athletics & Sports,Baby Store,Diner,Restaurant,Italian Restaurant,Nightclub
67,East York,0,Indian Restaurant,Yoga Studio,Supermarket,Liquor Store,Coffee Shop,Park,Gym,Pharmacy,Pizza Place,Burger Joint
77,North York,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Tea Room,Bakery,Food Court,Japanese Restaurant,Candy Store,Baseball Field
79,North York,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Women's Store,Tea Room,Bakery,Food Court,Japanese Restaurant,Candy Store,Baseball Field
81,North York,0,Furniture / Home Store,Coffee Shop,Miscellaneous Shop,Massage Studio,Bar,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Dim Sum Restaurant
101,West Toronto,0,Coffee Shop,Café,Breakfast Spot,Caribbean Restaurant,Furniture / Home Store,Burrito Place,Italian Restaurant,Climbing Gym,Convenience Store,Gym
116,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bakery,Deli / Bodega,Steakhouse,Seafood Restaurant,Gastropub


In [287]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,North York,1,Bank,Chinese Restaurant,Japanese Restaurant,Café,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
168,Central Toronto,1,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Rental Car Location,Dessert Shop,Bagel Shop,Diner,Salon / Barbershop,Sandwich Place
184,Downtown Toronto,1,Café,Coffee Shop,Restaurant,Bookstore,Bakery,Bar,Japanese Restaurant,Flower Shop,Italian Restaurant,Jazz Club
274,Etobicoke,1,Locksmith,Baseball Field,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
287,Etobicoke,1,Gym,Convenience Store,Burger Joint,Discount Store,Tanning Salon,Bakery,Thrift / Vintage Store,Sandwich Place,Supplement Shop,Fast Food Restaurant


In [288]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,2,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Restaurant,Bakery,Asian Restaurant,Gym,Hotel
96,North York,2,Park,Airport,Bus Stop,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
100,Downtown Toronto,2,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Gastropub,Italian Restaurant,Deli / Bodega,Gym,Sports Bar
115,Downtown Toronto,2,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bakery,Deli / Bodega,Steakhouse,Seafood Restaurant,Gastropub
117,North York,2,Park,Bakery,Basketball Court,Construction & Landscaping,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
139,North York,2,Baseball Field,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
159,West Toronto,2,Mexican Restaurant,Café,Grocery Store,Furniture / Home Store,Bookstore,Sandwich Place,Fast Food Restaurant,Flea Market,Speakeasy,Diner
185,Downtown Toronto,2,Café,Coffee Shop,Restaurant,Bookstore,Bakery,Bar,Japanese Restaurant,Flower Shop,Italian Restaurant,Jazz Club
219,Downtown Toronto,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Harbor / Marina
231,Etobicoke,2,Grocery Store,Pizza Place,Beer Store,Coffee Shop,Fried Chicken Joint,Video Store,Japanese Restaurant,Pharmacy,Fast Food Restaurant,Discount Store


In [289]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Etobicoke,3,Park,Pharmacy,Café,Pizza Place,Convenience Store,Liquor Store,Beer Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
108,Scarborough,3,Bakery,Bus Line,Soccer Field,Intersection,Park,Metro Station,Fast Food Restaurant,Convenience Store,Cosmetics Shop,College Stadium
133,York,3,Coffee Shop,Turkish Restaurant,Sandwich Place,Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Empanada Restaurant,Department Store
169,Central Toronto,3,Café,Sandwich Place,Coffee Shop,Pizza Place,American Restaurant,Liquor Store,Burger Joint,Jewish Restaurant,Flower Shop,Pub
250,Downtown Toronto,3,Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Pharmacy,Indian Restaurant,Pub,Bakery,Park,Café
284,Etobicoke,3,Gym,Convenience Store,Burger Joint,Discount Store,Tanning Salon,Bakery,Thrift / Vintage Store,Sandwich Place,Supplement Shop,Fast Food Restaurant


In [290]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,4,Fast Food Restaurant,Food & Drink Shop,Park,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,North York,4,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner
5,Downtown Toronto,4,Coffee Shop,Bakery,Pub,Café,Park,Theater,Breakfast Spot,Restaurant,Mexican Restaurant,Greek Restaurant
6,North York,4,Furniture / Home Store,Clothing Store,Coffee Shop,Boutique,Sporting Goods Shop,Miscellaneous Shop,Event Space,Vietnamese Restaurant,Accessories Store,Concert Hall
7,North York,4,Furniture / Home Store,Clothing Store,Coffee Shop,Boutique,Sporting Goods Shop,Miscellaneous Shop,Event Space,Vietnamese Restaurant,Accessories Store,Concert Hall
8,Queen's Park,4,Coffee Shop,Japanese Restaurant,Diner,Gym,Sushi Restaurant,Burger Joint,Park,Chinese Restaurant,Persian Restaurant,Creperie
10,Etobicoke,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Scarborough,4,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Gym
12,Scarborough,4,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Gym
14,North York,4,Japanese Restaurant,Gym / Fitness Center,Café,Basketball Court,Caribbean Restaurant,Baseball Field,Pool,Donut Shop,Discount Store,Dog Run
